In [1]:
from olinda.distillation import distill
from olinda.featurizer import MorganFeaturizer

2024-06-05 22:31:21.525909: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 22:31:21.525942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 22:31:21.526565: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-05 22:31:21.530377: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 22:31:22.195855: W tensorflow/compiler/tf2

Using TensorFlow backend


## A model to distill

You need a trained model for the distillation process. Here we are creating a simple pytorch model(untrained) for demo. A tensorflow model is also supported.

In [2]:
from typing import Any

import pytorch_lightning as pl
from torch import nn
from torch.nn import functional as F


class DemoModel(pl.LightningModule):
    """OlindaNet Zero: Slim(relatively) distillation network."""

    def __init__(
        self: "DemoModel"
    ) -> None:
        """Init."""
        super().__init__()
        #self.conv1 = nn.Conv2d(32, 32, kernel_size=3, padding=0, stride=3)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)

        

    def forward(self: "DemoModel", x: Any) -> Any:
        """Forward function.

        Args:
            x (Any): model input

        Returns:
            Any: model output
        """
        #x = self.conv1(x)
        #x = F.relu(x)

        # flattening while keeping the batch axis
        #x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x
###############

model = DemoModel()

## Distillation

Quickly test the distillation pipeline for your model with a small reference smiles dataset

In [3]:
#featurizer, student_model = distill(model, test=True, clean=True)
student_model = distill(model, clean=True, num_data=300) #An even smaller test set

Creating model output: 100%|██████████████████████| 9/9 [00:00<00:00, 25.78it/s]
2024-06-05 22:31:27.916838: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-06-05 22:31:27.916860: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: jason-Legion-5-15ACH6
2024-06-05 22:31:27.916865: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: jason-Legion-5-15ACH6
2024-06-05 22:31:27.916897: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 550.54.15
2024-06-05 22:31:27.916921: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 550.54.15
2024-06-05 22:31:27.916926: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 550.54.15


Reloading Tuner from ./trials/tuner0.json
Epoch 1/3
9/9 [==============================] - 1s 93ms/step - loss: 44.4484 - val_loss: 0.0206
Epoch 2/3
9/9 [==============================] - 1s 79ms/step - loss: 0.0117 - val_loss: 0.0036
Epoch 3/3
9/9 [==============================] - 1s 76ms/step - loss: 0.0076 - val_loss: 0.0026
Best epoch: 3
Epoch 1/3
9/9 [==============================] - 1s 89ms/step - loss: 29.2386 - val_loss: 0.5343
Epoch 2/3
9/9 [==============================] - 1s 76ms/step - loss: 0.1644 - val_loss: 0.0243
Epoch 3/3
9/9 [==============================] - 1s 76ms/step - loss: 0.0248 - val_loss: 0.0132


2024-06-05 22:31:33.655249: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-06-05 22:31:33.655337: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-06-05 22:31:33.697589: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-06-05 22:31:33.697701: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


In [4]:
x = MorganFeaturizer().featurize(["CCCOC"])
student_model(x)

[-0.049883883446455]